In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

from precision_recall_cutoff import precision_recall_cutoff

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'In_Class_Assignments/turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
turnover = pd.read_csv(file_content_stream)
turnover.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
## Changing sales and salary to dummies
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis = 1), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [3]:
## Defining the scaler
scaler = MinMaxScaler()

## Changing scale to 0-1
turnover[['number_project', 'average_montly_hours']] = scaler.fit_transform(turnover[['number_project', 'average_montly_hours']])

## BoxCox transformation
transformed_time_spend = boxcox(turnover['time_spend_company'])
turnover['time_spend_company'] = transformed_time_spend[0]

turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,0.0,0.285047,0.804651,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,0.6,0.775701,1.098118,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,1.0,0.822430,0.941381,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,0.6,0.593458,1.032330,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,0.0,0.294393,0.804651,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
## Engineering interactions
turnover['interaction_1'] = turnover['satisfaction_level'] * turnover['time_spend_company']
turnover['interaction_2'] = turnover['last_evaluation'] * turnover['promotion_last_5years']
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2
0,0.38,0.53,0.0,0.285047,0.804651,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0.305767,0.0
1,0.80,0.86,0.6,0.775701,1.098118,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0.878494,0.0
2,0.11,0.88,1.0,0.822430,0.941381,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0.103552,0.0
3,0.72,0.87,0.6,0.593458,1.032330,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0.743278,0.0
4,0.37,0.52,0.0,0.294393,0.804651,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0.297721,0.0


In [5]:
## Defining the input and target variables
X = turnover.drop(columns = 'left', axis = 1)
Y = turnover['left']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [6]:
X_train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2
12377,0.44,0.56,0.0,0.289720,0.804651,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0.354047,0.0
3424,0.71,0.56,0.4,0.663551,0.941381,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0.668380,0.0
8610,0.57,0.78,0.4,0.219626,0.804651,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.458651,0.0
4868,0.86,0.55,0.2,0.808411,0.567133,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.487734,0.0
843,0.66,0.47,1.0,0.280374,0.567133,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.374308,0.0


# Random Forest 1

In [8]:
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test)[:, 1]

## Changing likelihoods to labels
RF_label = precision_recall_cutoff(Y_test, RF_pred)

## Printing the classification report
print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.90      0.99      0.95      2286
           1       0.97      0.65      0.78       714

    accuracy                           0.91      3000
   macro avg       0.94      0.82      0.86      3000
weighted avg       0.92      0.91      0.91      3000



# Random Forest 2

In [9]:
## Removing interactions
X_train_new = X_train.drop(columns = ['interaction_1', 'interaction_2'], axis = 1)
X_test_new = X_test.drop(columns = ['interaction_1', 'interaction_2'], axis = 1)

RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_new, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test_new)[:, 1]

## Changing likelihoods to labels
RF_label = precision_recall_cutoff(Y_test, RF_pred)

## Printing the classification report
print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      2286
           1       0.95      0.66      0.78       714

    accuracy                           0.91      3000
   macro avg       0.92      0.82      0.86      3000
weighted avg       0.91      0.91      0.90      3000



In [ ]:
## Based on my results, I would use the second model to predict turnover because the 